## Employment Recommender Systems 
## Using Natural Language Processing

## Script 3 - Content-based recommender system

## Workbook Requirements

In [1]:
#!pip install -r '/work/requirements.txt'

In [2]:
# Uncategorized
import pandas as pd
import numpy as np
import os
import copy
import operator
from random import randint
from gensim import corpora, models
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from IPython.core.display import display, HTML
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

## Download and Load dataset

### Get the dataset:
##### Download manually:
- Download the dataset from google drive with this link:                                    
https://drive.google.com/drive/folders/1mWKlOg1z5Le6QrXbYJ4yGld4h6Dhrp0-?usp=sharing
- Save them in the same folder as this jupyter notebook file

##### OR use code as below:

In [3]:
# import model topics
job_data = pd.read_excel('clean_data.xlsx', index_col=0)

# import clean resume data
resume_1 = open("resume_software.txt", "r").readlines()
resume_1 = [x[:-1] for x in resume_1]

resume_2 = open("resume_aviation.txt", "r").readlines()
resume_2 = [x[:-1] for x in resume_2]

# import model topics
topics = pd.read_excel('topics_top2vec.xlsx', sheet_name = 1)

In [4]:
# inspect a few rows of clean job data
print(job_data.shape)
job_data.head()

(13342, 13)


,Position,Link,Company,Date,Description,Seniority_level,Employment_type,Department,Industry,Country,City,Language,Clean_description
0,English Teacher,https://at.linkedin.com/jobs/view/english-teac...,English in Action,2022-04-20,['English in Action is looking for qualified E...,NaN,Contract,NaN,NaN,Austria,Vienna,en,english action qualified english teachers pgc...
1,Office Assistant,https://at.linkedin.com/jobs/view/office-assis...,BAMBINIFASHION.COM,2022-04-29,['WHO WE ARE..BAMBINIFASHION.COM\xa0is one of ...,NaN,Full-time,NaN,NaN,Austria,Vienna,en,who are bambinifashion com europe premier ch...
2,Recruiter at Central European University,https://at.linkedin.com/jobs/view/recruiter-at...,join.com,2022-05-07,['Starting date: As soon as possible..Applicat...,Associate,Full-time,Human Resources,Higher Education,Austria,Vienna,en,starting date possible application deadline ...
3,Program Assistant,https://at.linkedin.com/jobs/view/program-assi...,Vienna Master of Arts in Applied Human Rights,2022-04-29,['The Vienna Master of Arts in Applied Human R...,NaN,Internship,NaN,NaN,Austria,Vienna,en,the vienna master arts applied human rights un...
4,Customer Service Representative,https://at.linkedin.com/jobs/view/customer-ser...,LIPOELASTIC Österreich,2022-05-06,['Customer Service & Office Assistant..Cosmeti...,NaN,Full-time,NaN,NaN,Austria,Vienna,en,customer service office assistant cosmetic ...


In [5]:
# inspect a few words of clean resume data
print(len(resume_1))
resume_1[0:150]

202


['software',
 'developer',
 'unlimited',
 'software',
 'developer',
 'unlimited',
 'worked',
 'small',
 'agile',
 'team',
 'startup',
 'environment',
 'prototype',
 'build',
 'mobile',
 'application',
 'daily',
 'responsibility',
 'included',
 'francisco',
 'birth',
 'place',
 'birth',
 'profile',
 'employment',
 'history',
 'education',
 'skill',
 'elliot',
 'alderson',
 'software',
 'developer',
 'main',
 'francisco',
 'united',
 'state',
 'rozenboomchantal',
 'francisco',
 'nationality',
 'driving',
 'license',
 'passionate',
 'software',
 'engineer',
 'year',
 'professional',
 'building',
 'application',
 'proficient',
 'development',
 'stack',
 'software',
 'developer',
 'johnson',
 'johnson',
 'johnson',
 'johnson',
 'fortune',
 'medical',
 'device',
 'manufacturing',
 'company',
 'software',
 'developer',
 'work',
 'ecommerce',
 'platform',
 'agile',
 'environment',
 'daily',
 'responsibility',
 'include',
 'francisco',
 'participating',
 'daily',
 'stand',
 'meeting',
 'scrum',

In [6]:
# inspect a few words of clean resume data
print(len(resume_2))
resume_2[0:150]

191


['sidney',
 'pereira',
 'flight',
 'attendant',
 'year',
 'domestic',
 'charter',
 'commercial',
 'flight',
 'consistently',
 'praised',
 'passenger',
 'exceptional',
 'customer',
 'service',
 'certified',
 'trained',
 'emergency',
 'procedure',
 'technique',
 'flight',
 'attendant',
 'british',
 'airway',
 'london',
 'complete',
 'hour',
 'domestic',
 'flight',
 'boeing',
 'airbus',
 'commercial',
 'jet',
 'holding',
 'passenger',
 'assist',
 'passenger',
 'stow',
 'luggage',
 'saving',
 'cargo',
 'space',
 'efficient',
 'stowing',
 'technique',
 'ensure',
 'adherence',
 'company',
 'regulation',
 'mediation',
 'training',
 'defuse',
 'situation',
 'preventing',
 'reaching',
 'emergency',
 'level',
 'strictly',
 'follow',
 'regulation',
 'recommendation',
 'infection',
 'prevention',
 'procedure',
 'associate',
 'art',
 'hospitality',
 'customer',
 'service',
 'national',
 'university',
 'flight',
 'attendant',
 'lufthansa',
 'berlin',
 'completed',
 'hour',
 'domestic',
 'commercial'

In [7]:
# inspect a few rows of clean topics data
print(topics.shape)
topics.head()

(70, 51)


,topic_name,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,Accounting,reconciliations,accounting,ledger,accountant,finance,payable,invoices,reconciliation,receivable,...,quarterly,taxes,treasury,reputable,discrepancies,expenses,popis,expense,controlling,budgeting
1,Advertising,sections,date,information,notifications,recommend,campaign,note,specification,title,...,achievements,hospital,called,nhs,take,dates,university,formed,job,duration
2,NGOs,unhcr,humanitarian,donor,donors,incumbent,unicef,irc,secretariat,context,...,political,policies,implementation,evaluation,individually,capacity,grant,administrative,policy,description
3,AML,aml,laundering,kyc,sanctions,crime,diligence,anti,compliance,money,...,regulated,economic,fintech,interpretation,know,analyst,barclays,flex,help,legislative
4,App Developer,appen,annotation,tesco,crowd,text,earn,smartphone,images,speech,...,roleyou,everyones,income,household,surveys,web,enjoy,thank,skillshigh,submit


In [8]:
# dropping rows without "Description"
topics.dropna(subset=['topic_name'], inplace=True)

In [9]:
# group words in topic together
def merge_topic_data(model_topics):
    # group each of the 50 most relevant words to each topics together into one string
    all_words_in_topic = []
    for index, row in model_topics.iloc[:, 1:].iterrows():
        all_words_in_topic.append(row.str.cat(sep=' '))
    
    return all_words_in_topic

all_words_in_topic = merge_topic_data(topics)

In [10]:
topics = pd.DataFrame({'topic_name':topics.iloc[:, 0],'words_in_topic':all_words_in_topic})

In [11]:
# Find ID of topics that received the same label
topics.loc[(topics['topic_name'] == "Hospitality") | (topics['topic_name'] == "Consulting")]

,topic_name,words_in_topic
15,Consulting,consulting deloitte pwc advisory tax clients a...
16,Consulting,ote uncapped earning earnings commission incen...
35,Hospitality,hotel guests guest hospitality hotels suites r...
36,Hospitality,travelers expedia travel tours tourism tour de...


In [12]:
# create new rows by combining the rows with duplicate topic name together

consulting = {"topic_name": "Consulting", "words_in_topic": 
              topics.iloc[15]['words_in_topic'] + topics.iloc[16]['words_in_topic']}
hospitality = {"topic_name": "Hospitality", "words_in_topic": 
              topics.iloc[35]['words_in_topic'] + topics.iloc[36]['words_in_topic']}

In [13]:
# drop duplicates (but contain the combined)
topics.drop_duplicates(subset='topic_name', keep = False, inplace=True)

# append/concat combined rows to topics df
topics = topics.append(consulting, ignore_index=True)
topics = topics.append(hospitality, ignore_index=True)

In [14]:
# add resume to the bottom of the dataframe of topics

resume_row_1 = {"topic_name": "Resume", "words_in_topic": " ".join(resume_1)}
resume_row_2 = {"topic_name": "Resume", "words_in_topic": " ".join(resume_2)}

# create to separate topic dataset for each sample resume (we have two)
topics_1 = copy.deepcopy(topics)
topics_2 = copy.deepcopy(topics)

topics_1 = topics_1.append(resume_row_1, ignore_index=True)
topics_2 = topics_2.append(resume_row_2, ignore_index=True)

In [15]:
topics_1

,topic_name,words_in_topic
0,Accounting,reconciliations accounting ledger accountant f...
1,Advertising,sections date information notifications recomm...
2,NGOs,unhcr humanitarian donor donors incumbent unic...
3,AML,aml laundering kyc sanctions crime diligence a...
4,App Developer,appen annotation tesco crowd text earn smartph...
...,...,...
57,Translation,translation translator translators localizatio...
58,Underwriting,underwriting brokers axa claims renewal insura...
59,Consulting,consulting deloitte pwc advisory tax clients a...
60,Hospitality,hotel guests guest hospitality hotels suites r...


In [16]:
# the second resume is now in the bottom
topics_2

,topic_name,words_in_topic
0,Accounting,reconciliations accounting ledger accountant f...
1,Advertising,sections date information notifications recomm...
2,NGOs,unhcr humanitarian donor donors incumbent unic...
3,AML,aml laundering kyc sanctions crime diligence a...
4,App Developer,appen annotation tesco crowd text earn smartph...
...,...,...
57,Translation,translation translator translators localizatio...
58,Underwriting,underwriting brokers axa claims renewal insura...
59,Consulting,consulting deloitte pwc advisory tax clients a...
60,Hospitality,hotel guests guest hospitality hotels suites r...


## Create a metric for measuring similarity

In [17]:
def cosine_similarity(data, embedding):
    if embedding == "TFIDF":
        tfidf = TfidfVectorizer()
        matrix = tfidf.fit_transform(data)
    
    elif embedding == "CountVec":
        vectorizer = CountVectorizer()
        matrix = vectorizer.fit_transform(data)

    else:
        print("Unexpected argument")
    
    # calculate cosine similarity
    cosine_sim = linear_kernel(matrix, matrix)
    
    return cosine_sim

# Topic Recommendations 

In [18]:
def get_recommendations_topics(topics):
    
    # we use CountVec as word occurrence is more important than word frequency given that words 
    # never appear twice in the topics
    cosine_sim = cosine_similarity(topics['words_in_topic'], 'CountVec')
    
    # creates a similarity score of every description to the resume
    similarity_scores = sorted(list(enumerate(cosine_sim[-1])), key=lambda x: x[1], reverse=True)
    
    # finds the index of the 20 most similar descriptions
    job_indices = [i[0] for i in similarity_scores[2:20]]
    # output recommendation
    print("Based on your experience we believe you are best suited for the following fields:")
    topic_recommendations = topics.iloc[job_indices]
    topic_recommendations = topic_recommendations.reset_index()
    print(str(list(topic_recommendations["topic_name"][0:5])))
    return topic_recommendations

In [19]:
# return top 3 most similar topics to CV. These can be interpreted as fields that 
# would suit the  given CV

topic_recommendations_resume_1 = get_recommendations_topics(topics_1)
topic_recommendations_resume_2 = get_recommendations_topics(topics_2)

Based on your experience we believe you are best suited for the following fields:
['IT Services', 'Data Science', 'Logistics', 'Biotech', 'Engineering']
Based on your experience we believe you are best suited for the following fields:
['Bilingual ', 'Hospitality', 'Commerce', 'NGOs', 'Freight']


In [20]:
topic_recommendations_resume_1

,index,topic_name,words_in_topic
0,36,IT Services,hardware desktop troubleshooting windows troub...
1,19,Data Science,python statistical statistics scientist sql al...
2,39,Logistics,supply chain inventory demand planner johnson ...
3,11,Biotech,laboratory chemistry gmp laboratories chemical...
4,21,Engineering,mechanical drawings engineer cad electrical en...
5,1,Advertising,sections date information notifications recomm...
6,54,Tech Architecture,user translating functional methodologies goog...
7,13,Civil Engineering,aecom environmental water consultancy construc...
8,27,Gig Jobs,grocery startup wolt curve hierarchies founder...
9,28,Government,police government hmrc vetting eea directorate...


In [21]:
topic_recommendations_resume_2

,index,topic_name,words_in_topic
0,10,Bilingual,lisbon rotating portuguese skillsteam italian ...
1,60,Hospitality,hotel guests guest hospitality hotels suites r...
2,14,Commerce,store merchandising vuitton luxury louis ambas...
3,2,NGOs,unhcr humanitarian donor donors incumbent unic...
4,25,Freight,freight export forwarding shipments import cus...
5,39,Logistics,supply chain inventory demand planner johnson ...
6,42,Procurement,procurement purchasing supplier suppliers purc...
7,21,Engineering,mechanical drawings engineer cad electrical en...
8,27,Gig Jobs,grocery startup wolt curve hierarchies founder...
9,32,Higher education,university universitys college teaching studen...


# Job recommendations based on similarity to topic

In [22]:
# list with all jobs per top topic
topic_1_resume_1 = copy.deepcopy(job_data)
topic_2_resume_1 = copy.deepcopy(job_data)
topic_3_resume_1 = copy.deepcopy(job_data)

topic_1_resume_2 = copy.deepcopy(job_data)
topic_2_resume_2 = copy.deepcopy(job_data)
topic_3_resume_2 = copy.deepcopy(job_data)

topic_1_resume_1.loc[-1] = [topic_recommendations_resume_1['topic_name'][0],"Not Applicable","Not Applicable","Not Applicable",
                   "Not Applicable","Not Applicable","Not Applicable","Not Applicable",
                   "Not Applicable","Not Applicable","Not Applicable","Not Applicable",
                  topic_recommendations_resume_1["words_in_topic"][0]]
topic_2_resume_1.loc[-1] = [topic_recommendations_resume_1['topic_name'][1],"Not Applicable","Not Applicable","Not Applicable",
                   "Not Applicable","Not Applicable","Not Applicable","Not Applicable",
                   "Not Applicable","Not Applicable","Not Applicable","Not Applicable",
                  topic_recommendations_resume_1["words_in_topic"][1]]
topic_3_resume_1.loc[-1] = [topic_recommendations_resume_1['topic_name'][2],"Not Applicable","Not Applicable","Not Applicable",
                   "Not Applicable","Not Applicable","Not Applicable","Not Applicable",
                   "Not Applicable","Not Applicable","Not Applicable","Not Applicable",
                  topic_recommendations_resume_1["words_in_topic"][3]]

topic_1_resume_2.loc[-1] = [topic_recommendations_resume_2['topic_name'][0],"Not Applicable","Not Applicable","Not Applicable",
                   "Not Applicable","Not Applicable","Not Applicable","Not Applicable",
                   "Not Applicable","Not Applicable","Not Applicable","Not Applicable",
                  topic_recommendations_resume_2["words_in_topic"][0]]
topic_2_resume_2.loc[-1] = [topic_recommendations_resume_2['topic_name'][1],"Not Applicable","Not Applicable","Not Applicable",
                   "Not Applicable","Not Applicable","Not Applicable","Not Applicable",
                   "Not Applicable","Not Applicable","Not Applicable","Not Applicable",
                  topic_recommendations_resume_2["words_in_topic"][1]]
topic_3_resume_2.loc[-1] = [topic_recommendations_resume_2['topic_name'][2],"Not Applicable","Not Applicable","Not Applicable",
                   "Not Applicable","Not Applicable","Not Applicable","Not Applicable",
                   "Not Applicable","Not Applicable","Not Applicable","Not Applicable",
                  topic_recommendations_resume_2["words_in_topic"][3]]

topics_list_resume_1 = [topic_1_resume_1, topic_2_resume_1, topic_3_resume_1]
topics_list_resume_2 = [topic_1_resume_2, topic_2_resume_2, topic_3_resume_2]

In [23]:
def get_recommendations_jobs(topics_list):
    
    # output recommendation
    print("We recommend jobs within the following fields:")
    
    # generate recommendations for each topic/field
    for index, topic in enumerate(topics_list):
        print(topic["Position"][-1])
        
        # create cosine matrix
        cosine_sim = cosine_similarity(topic.iloc[: , -1].values.astype(str), 'TFIDF')

        # creates a similarity score of every description to the resume
        similarity_scores = sorted(list(enumerate(cosine_sim[-1])), key=lambda x: x[1], reverse=True)

        # finds the index of the 20 most similar descriptions
        job_indices_lite = [i[0] for i in similarity_scores[2:10]]
        job_indices = [i[0] for i in similarity_scores[2:250]]
        
        # save recommendations (end document name with random integer to prevent overwrite issues)
        topic[["Position","Company", "City", "Link","Clean_description"]].iloc[job_indices].to_excel(
            os.getcwd()+'\\recommendations_based_on_topic_' + str(index) + '____' + str(randint(0, 1000)) + '.xlsx')
        
        # display results
        display(HTML(topic[["Position","Company", "City", "Link"]].iloc[job_indices_lite].to_html(render_links=True, escape=False)))

In [24]:
# Note that these recommendations do not use the resume to recommend jobs
# they are simply recommended because they are jobs that fit the fields that the person
# is best suited for

get_recommendations_jobs(topics_list_resume_1)

We recommend jobs within the following fields:
IT Services


,Position,Company,City,Link
780,Support Engineer,DeRisk Technologies,Graz,https://at.linkedin.com/jobs/view/support-engineer-on-demand-at-derisk-technologies-3059941022?refId=CXGuGeoW%2FenLLyTTkghuMg%3D%3D&trackingId=xC5HhW6n3lmE%2B7UbH%2BhJVg%3D%3D&position=7&pageNum=31&trk=public_jobs_jserp-result_search-card
9541,It Support Specialist In Italian/English,The Sales Centre,Italy,https://it.linkedin.com/jobs/view/it-support-specialist-in-italian-english-at-the-sales-centre-3043189983?refId=7eHKWT4%2F0SiL8px%2BPSPJkw%3D%3D&trackingId=9tWBsuzC0A3xDpwhLLOGkg%3D%3D&position=7&pageNum=4&trk=public_jobs_jserp-result_search-card
11416,IT Technician,Blue Arrow,Runcorn,https://uk.linkedin.com/jobs/view/it-technician-at-blue-arrow-3039359048?refId=kpoVOQc1V%2Frmdozvp2NYjA%3D%3D&trackingId=b%2BZn6IBsUGCJLjidEfrCNQ%3D%3D&position=1&pageNum=32&trk=public_jobs_jserp-result_search-card
9138,General IT Support Technician,Cognizant,Dublin,https://ie.linkedin.com/jobs/view/general-it-support-technician-l2-at-cognizant-3062153362?refId=OCfJE9ugVPu9f19b790rNA%3D%3D&trackingId=BHcPT445DpeCZcQn5neNaQ%3D%3D&position=18&pageNum=12&trk=public_jobs_jserp-result_search-card
11122,IT Service Manager,Mission Viejo Auto Collision,Liverpool,https://uk.linkedin.com/jobs/view/it-service-manager-at-mission-viejo-auto-collision-3065864955?refId=vWchDxgrVCb0EeZOox8Zog%3D%3D&trackingId=Yo7yjebNuqCjk1527XNpNg%3D%3D&position=7&pageNum=20&trk=public_jobs_jserp-result_search-card
11060,IT Support Technician,Adecco,Runcorn,https://uk.linkedin.com/jobs/view/it-support-technician-at-adecco-3044366933?refId=rswGnd3NkGUlbYCIL8jwpA%3D%3D&trackingId=XmpqbqQPqMUaBxzeeN6OhA%3D%3D&position=20&pageNum=17&trk=public_jobs_jserp-result_search-card
11070,IT Service Manager,Michael Page,Liverpool,https://uk.linkedin.com/jobs/view/it-service-manager-at-michael-page-3061502126?refId=iQSLMICs5QKtF%2FlmBRlIBg%3D%3D&trackingId=Ro%2FZacUD2ThcBj37qyHmQA%3D%3D&position=5&pageNum=18&trk=public_jobs_jserp-result_search-card
5940,Systems Administrator,The American University of Paris,Paris,https://fr.linkedin.com/jobs/view/systems-administrator-at-the-american-university-of-paris-3060807656?refId=TtzchR6DrRj7HHq7vQmg2A%3D%3D&trackingId=86doxqGL49yhe22eWqBr7Q%3D%3D&position=3&pageNum=2&trk=public_jobs_jserp-result_search-card


Data Science


,Position,Company,City,Link
21710,Data Scientist,Randstad,Geneva,https://ch.linkedin.com/jobs/view/data-scientist-at-randstad-3060269726?refId=inCmMzyUQ5pkrXBKqoCRIg%3D%3D&trackingId=VuPJgcbYcMyA27pAxaz9Cg%3D%3D&position=6&pageNum=31&trk=public_jobs_jserp-result_search-card
579,Quantitative Credit Risk Analyst,Raiffeisen Bank International AG,Austria,https://at.linkedin.com/jobs/view/quantitative-credit-risk-analyst-f-m-x-at-raiffeisen-bank-international-ag-3049267963?refId=AL6UBqNsb4htW40PXBMgjw%3D%3D&trackingId=gBwAD27yEeMLm0MombUczQ%3D%3D&position=6&pageNum=23&trk=public_jobs_jserp-result_search-card
4610,"Data Scientist / Machine Learning Engineer, FinTech Remote",ViaBill,Copenhagen,https://dk.linkedin.com/jobs/view/data-scientist-machine-learning-engineer-fintech-remote-at-viabill-3025850198?refId=%2BhqLWrNhjkpGFJeauPpcPw%3D%3D&trackingId=D0PkK84LQOdH%2BjajcaSOVg%3D%3D&position=21&pageNum=25&trk=public_jobs_jserp-result_search-card
8197,Junior Data Scientist,Citrix,Athens,https://gr.linkedin.com/jobs/view/junior-data-scientist-remote-at-citrix-3016704265?refId=pPL8ce0aPF%2F3lGJPE6HqZg%3D%3D&trackingId=iuw8L4jtudCx3rjXp3qyBA%3D%3D&position=23&pageNum=14&trk=public_jobs_jserp-result_search-card
15976,Quantitative Analyst,Greenfact,Oslo,https://no.linkedin.com/jobs/view/quantitative-analyst-at-greenfact-2964572918?refId=DI85qzmFT8pUX3fV%2Bh8qjA%3D%3D&trackingId=Mr%2BewMjq2hNyxBMLnpHyzA%3D%3D&position=4&pageNum=30&trk=public_jobs_jserp-result_search-card
12215,Business Intelligence Analyst,Lawrence Harvey,London,https://uk.linkedin.com/jobs/view/business-intelligence-analyst-at-lawrence-harvey-3051236355?refId=EGJRTmHJA264GnKMkye0xA%3D%3D&trackingId=GA2akKgLstmZYmkAsTZg2A%3D%3D&position=1&pageNum=24&trk=public_jobs_jserp-result_search-card
8539,Machine Learning Engineer / Data Analyst,Elxis Group,Athens,https://gr.linkedin.com/jobs/view/machine-learning-engineer-data-analyst-at-elxis-group-3035944467?refId=B%2Fs5w0m3CS0ViSOrpVvJHg%3D%3D&trackingId=20YqCKlF3%2FF0j2cqhLUyNw%3D%3D&position=16&pageNum=28&trk=public_jobs_jserp-result_search-card
6946,Business Insights & Analytics,Barclays,Glasgow,https://uk.linkedin.com/jobs/view/business-insights-analytics-analyst-at-barclays-3049231705?refId=r0wGCAGJaob%2Faz20Sn%2FMKQ%3D%3D&trackingId=KGXfYoTLuKE7GvYmMk0aIw%3D%3D&position=17&pageNum=4&trk=public_jobs_jserp-result_search-card


Logistics


,Position,Company,City,Link
4857,Analytical Scientist for Drug Product Development,Novo Nordisk,Måløv,https://dk.linkedin.com/jobs/view/analytical-scientist-for-drug-product-development-in-r-d-at-novo-nordisk-3058110779?refId=cuSq8UQDakhhdwHtCTeSNg%3D%3D&trackingId=jWE9hGfyboOSHmgK2dkbLw%3D%3D&position=19&pageNum=35&trk=public_jobs_jserp-result_search-card
5864,R&D Laboratory Technician,Thermo Fisher Scientific,Vantaa,https://fi.linkedin.com/jobs/view/r-d-laboratory-technician-at-thermo-fisher-scientific-3043105088?refId=%2Bux%2BjfCKIypaISdL7lSIBQ%3D%3D&trackingId=%2BstPWVx2WTDjfPPsQWopIw%3D%3D&position=1&pageNum=39&trk=public_jobs_jserp-result_search-card
3967,Laboratory Chemist,Roemex Ltd,Kongens Lyngby,https://dk.linkedin.com/jobs/view/laboratory-chemist-at-roemex-ltd-3057196402?refId=0uWgexKmELgZ0Sciwk7tPw%3D%3D&trackingId=jmZxLi2jfeAaQOAx5UZDbQ%3D%3D&position=2&pageNum=0&trk=public_jobs_jserp-result_search-card
8879,QC Chemistry Analyst,Cpl,Dublin,https://ie.linkedin.com/jobs/view/qc-chemistry-analyst-pfizer-at-cpl-3051207732?refId=YJr1hYGfVJTNK%2BIvws3R5Q%3D%3D&trackingId=C2vUvc8sQFEU26G5NwsncQ%3D%3D&position=9&pageNum=2&trk=public_jobs_jserp-result_search-card
7513,Associate Scientist,Merck Group,Glasgow,https://uk.linkedin.com/jobs/view/associate-scientist-2-at-merck-group-3037954668?refId=ziqlIMDr6OJp4MicLPaIJA%3D%3D&trackingId=ZVDk1NI3UqJszk68iNPnDw%3D%3D&position=11&pageNum=27&trk=public_jobs_jserp-result_search-card
13882,Pharmaceutical Scientist,Coriolis Pharma,Planegg,https://de.linkedin.com/jobs/view/pharmaceutical-scientist-project-manager-gmp-bioanalytics-m-f-d-coriolis-pharma-research-gmbh-at-coriolis-pharma-3043262351?refId=bVJj1Vru3WOsiL0EZO1F%2Fg%3D%3D&trackingId=EZrrY%2Bojzf2N4%2BXlw9YfjA%3D%3D&position=20&pageNum=15&trk=public_jobs_jserp-result_search-card
7444,QC Analyst,SGS,Glasgow,https://uk.linkedin.com/jobs/view/qc-analyst-at-sgs-3020724764?refId=7jQTkIKxmu3p8PHABVm%2FpA%3D%3D&trackingId=E0Vw7ifVo4YwIZwyyrTliQ%3D%3D&position=17&pageNum=24&trk=public_jobs_jserp-result_search-card
14680,Science Affairs Liason,Oxford Global Resources,Oosterhout,https://nl.linkedin.com/jobs/view/science-affairs-liason-at-oxford-global-resources-3092691671?refId=A2%2BWsAv0Q4DKyfo557Zv7g%3D%3D&trackingId=Eim4505S3j3HAIAZJsN1tA%3D%3D&position=19&pageNum=17&trk=public_jobs_jserp-result_search-card


In [25]:
# Note that these recommendations do not use the resume to recommend jobs
# they are simply recommended because they are jobs that fit the fields that the person
# is best suited for

get_recommendations_jobs(topics_list_resume_2)

We recommend jobs within the following fields:
Bilingual 


,Position,Company,City,Link
17178,English Customer Advisor,Cross Border Talents,Lisbon,https://pt.linkedin.com/jobs/view/english-customer-advisor-lisbon-portugal-at-cross-border-talents-2930303939?refId=7hfVBgx5YXz5u14aJXWyVg%3D%3D&trackingId=Ap%2BVvxCoCdmd63CATTPJdA%3D%3D&position=10&pageNum=12&trk=public_jobs_jserp-result_search-card
16877,Spanish Customer Service Representatives,Cross Border Talents,Lisbon Metropolitan Area,https://pt.linkedin.com/jobs/view/spanish-customer-service-representatives-at-cross-border-talents-3083629131?refId=V67Ep2vuKQ7Xp9tI03i%2Frw%3D%3D&trackingId=sLYRt2WufjOjk7xdoyDNZQ%3D%3D&position=7&pageNum=0&trk=public_jobs_jserp-result_search-card
16972,English Customer Support Agent,Cross Border Talents,Lisbon,https://pt.linkedin.com/jobs/view/english-customer-support-agent-lisbon-at-cross-border-talents-2921690096?refId=dyqhQnCOh3qhbH%2F9tqkXgw%3D%3D&trackingId=rMt5S%2Bc%2BOG%2BzgkrjraR3wQ%3D%3D&position=2&pageNum=4&trk=public_jobs_jserp-result_search-card
16973,Responsável Administrativo e Financeiro,Evollu,Leiria,https://pt.linkedin.com/jobs/view/respons%C3%A1vel-administrativo-e-financeiro-at-evollu-3092696591?refId=dyqhQnCOh3qhbH%2F9tqkXgw%3D%3D&trackingId=A%2B7KZJUw2mTNHLzK7ttmOQ%3D%3D&position=3&pageNum=4&trk=public_jobs_jserp-result_search-card
17076,Content development and on,Secretplaces,Cascais,https://pt.linkedin.com/jobs/view/content-development-and-on-line-marketing-internship-native-english-speaker-at-secretplaces-3051166471?refId=sPun91n9tIjN4Rn%2F%2B3W8dw%3D%3D&trackingId=ttYV7JC1YoML2Uxun6MIEA%3D%3D&position=8&pageNum=8&trk=public_jobs_jserp-result_search-card
17152,Proofreader,VdA Vieira de Almeida,Lisbon,https://pt.linkedin.com/jobs/view/proofreader-at-vda-vieira-de-almeida-3081923341?refId=PN%2Btz0tOkDSh3%2BXsmFwZlA%3D%3D&trackingId=rNVLI9U3KEfGu%2BOqSEOzew%3D%3D&position=9&pageNum=11&trk=public_jobs_jserp-result_search-card
17048,International Recruiter,Cross Border Talents,Lisbon,https://pt.linkedin.com/jobs/view/international-recruiter-at-cross-border-talents-3065863570?refId=ym7stkEgWjhLA%2BzvATrKIA%3D%3D&trackingId=FuFEOM4ARQ%2Bh%2Fo9qKrZmYA%3D%3D&position=3&pageNum=7&trk=public_jobs_jserp-result_search-card
16898,Customer Service Specialist,Cross Border Talents,Lisbon Metropolitan Area,https://pt.linkedin.com/jobs/view/customer-service-specialist-at-cross-border-talents-3090141072?refId=R%2BwT2jAeEk3nCgT%2BapEWeA%3D%3D&trackingId=DfbkA5oLHxxZyb7V2NRQhg%3D%3D&position=3&pageNum=1&trk=public_jobs_jserp-result_search-card


Hospitality


,Position,Company,City,Link
10726,Receptionist,Hilton,Greater Liverpool Area,https://uk.linkedin.com/jobs/view/receptionist-at-hilton-3044804504?refId=JmYhXKdV1POO%2FZ6arqXBvA%3D%3D&trackingId=TCLjJUimT%2FU4aQbKrSBrwQ%3D%3D&position=11&pageNum=4&trk=public_jobs_jserp-result_search-card
1897,Night Manager,Hilton,Antwerpområdet,https://be.linkedin.com/jobs/view/night-manager-emea-at-hilton-3047640455?refId=YOHCJ9WNwJpgJ70I7CeRng%3D%3D&trackingId=Hf7c%2FlMKLtp0uZFXjbNSUQ%3D%3D&position=10&pageNum=36&trk=public_jobs_jserp-result_search-card
20242,Reservations Agent,Hilton,Greater Stockholm Metropolitan Area,https://se.linkedin.com/jobs/view/reservations-agent-at-hilton-3064829913?refId=I0yHfR32DIjnHh%2BjkHw1rA%3D%3D&trackingId=Nu5Pw0gZazryxqZAqLzgTQ%3D%3D&position=13&pageNum=12&trk=public_jobs_jserp-result_search-card
7696,"Groups, Reservations Coordinator",Radisson Hotel Group,Glasgow,https://uk.linkedin.com/jobs/view/groups-reservations-coordinator-at-radisson-hotel-group-3053395590?refId=r9kHmNhQh%2BRp9RYtP2bM7A%3D%3D&trackingId=IxRVXMbw%2F6KD33SC%2B%2FkymA%3D%3D&position=19&pageNum=34&trk=public_jobs_jserp-result_search-card
4073,Night receptionist/auditor,Radisson Hotel Group,Copenhagen,https://dk.linkedin.com/jobs/view/night-receptionist-auditor-part-time-at-radisson-hotel-group-3033715896?refId=U7fG90GPg7IFA%2FYMnzKiYA%3D%3D&trackingId=aowWom7OhL5lSHG8jHtVTA%3D%3D&position=8&pageNum=4&trk=public_jobs_jserp-result_search-card
3149,Front Office Supervisor,Hilton,Prague,https://cz.linkedin.com/jobs/view/front-office-supervisor-at-hilton-3055857099?refId=WdkVOpyAgdEkb9jbSXdpPA%3D%3D&trackingId=bMWkHAGrBxBViE6nQpoGfQ%3D%3D&position=9&pageNum=6&trk=public_jobs_jserp-result_search-card
7679,Reservations Coordinator,Radisson Hotel Group,Glasgow,https://uk.linkedin.com/jobs/view/reservations-coordinator-at-radisson-hotel-group-3032428231?refId=r9kHmNhQh%2BRp9RYtP2bM7A%3D%3D&trackingId=mJw%2BnvrtwC1DmzFFGaNcfQ%3D%3D&position=2&pageNum=34&trk=public_jobs_jserp-result_search-card
15294,Receptionist,Radisson Hotel Group,Gardermoen,https://no.linkedin.com/jobs/view/receptionist-at-radisson-hotel-group-3030946993?refId=57i3CbRDYEE9jc0VCR%2BlGQ%3D%3D&trackingId=wrHMOA8KW5INwXZ1f0oDyA%3D%3D&position=22&pageNum=2&trk=public_jobs_jserp-result_search-card


Commerce


,Position,Company,City,Link
21799,Humanitarian Affairs Officer.P,United Nations OCHA,Geneva,https://ch.linkedin.com/jobs/view/humanitarian-affairs-officer-p3-at-united-nations-ocha-3063012495?refId=KhYzrRJj21%2FIZsLjPtUKkg%3D%3D&trackingId=pq7g3%2BoieKslxGHtjo3WgA%3D%3D&position=20&pageNum=34&trk=public_jobs_jserp-result_search-card
16281,Child Protection Associate G,"UNHCR, the UN Refugee Agency",Warszawa,https://pl.linkedin.com/jobs/view/child-protection-associate-g6-36343-at-unhcr-the-un-refugee-agency-3049291517?refId=tnzbkvuSpenGJdXUA7LFgQ%3D%3D&trackingId=08GeGoNY4hy5e4u5M9alzQ%3D%3D&position=9&pageNum=5&trk=public_jobs_jserp-result_search-card
16284,Inter,"UNHCR, the UN Refugee Agency",Warszawa,https://pl.linkedin.com/jobs/view/inter-agency-coordination-assc-g6-36404-at-unhcr-the-un-refugee-agency-3049292706?refId=tnzbkvuSpenGJdXUA7LFgQ%3D%3D&trackingId=0gtCt79Em1bR1SGHEq%2FSMw%3D%3D&position=12&pageNum=5&trk=public_jobs_jserp-result_search-card
21074,Assistant Representative,"UNHCR, the UN Refugee Agency",Geneva,https://ch.linkedin.com/jobs/view/assistant-representative-administration-at-unhcr-the-un-refugee-agency-3059470840?refId=VGFNWwGxKgguI6uELb3JYQ%3D%3D&trackingId=%2FFlsRs3tiHQ8CbkB4LhR1Q%3D%3D&position=20&pageNum=5&trk=public_jobs_jserp-result_search-card
16422,Assistant Inter,"UNHCR, the UN Refugee Agency",Lublin,https://pl.linkedin.com/jobs/view/assistant-inter-agency-coordination-officer-36355-at-unhcr-the-un-refugee-agency-3049289833?refId=LAr2TJmzDXCaMN8ZrVj1%2FA%3D%3D&trackingId=bkrhoRMnYZqfH50sZi2Y4w%3D%3D&position=25&pageNum=10&trk=public_jobs_jserp-result_search-card
16626,Assistant HR Officer,"UNHCR, the UN Refugee Agency",Warszawa,https://pl.linkedin.com/jobs/view/assistant-hr-officer-36325-at-unhcr-the-un-refugee-agency-3049292489?refId=%2FnNpX6IijQ61ekb4T3gpGw%3D%3D&trackingId=xx4cksKnolP5pjn5T3H1pQ%3D%3D&position=5&pageNum=19&trk=public_jobs_jserp-result_search-card
16357,HR Associate,"UNHCR, the UN Refugee Agency",Krakow,https://pl.linkedin.com/jobs/view/hr-associate-36376-at-unhcr-the-un-refugee-agency-3049293581?refId=yymSaaJwvkU5m9rBVGFsqw%3D%3D&trackingId=vygVVz%2Fbsr8xlcUDe21UEg%3D%3D&position=10&pageNum=8&trk=public_jobs_jserp-result_search-card
4832,"Logistics Manager , P, FT, Copenhagen, Denmark #",UNICEF,Copenhagen,https://dk.linkedin.com/jobs/view/logistics-manager-emergency-preparedness-p4-ft-copenhagen-denmark-%2386201-at-unicef-3044327832?refId=lV6anp06hc1%2B0EBAk%2F1vPQ%3D%3D&trackingId=yPVcoyUawEu%2FUPJ2gcsHxw%3D%3D&position=19&pageNum=34&trk=public_jobs_jserp-result_search-card


Next we want to calculate the similarity between a resume and the jobs in a specific topic

In [26]:
# import the topics we just saved locally
topic_1_resume_1 = pd.read_excel('recommendations_based_on_topic_0_resume_1.xlsx', index_col=0)
topic_2_resume_1 = pd.read_excel('recommendations_based_on_topic_1_resume_1.xlsx', index_col=0)
topic_3_resume_1 = pd.read_excel('recommendations_based_on_topic_2_resume_1.xlsx', index_col=0)

print(topic_1_resume_1.shape)
print(topic_2_resume_1.shape)
print(topic_3_resume_1.shape)

topic_1_resume_2 = pd.read_excel('recommendations_based_on_topic_0_resume_2.xlsx', index_col=0)
topic_2_resume_2 = pd.read_excel('recommendations_based_on_topic_1_resume_2.xlsx', index_col=0)
topic_3_resume_2 = pd.read_excel('recommendations_based_on_topic_2_resume_2.xlsx', index_col=0)

print(topic_1_resume_2.shape)
print(topic_2_resume_2.shape)
print(topic_3_resume_2.shape)

(248, 5)
(248, 5)
(248, 5)
(248, 5)
(248, 5)
(248, 5)


In [27]:
# inspect one of them
topic_1_resume_1.head()

,Position,Company,City,Link,Clean_description
780,Support Engineer,DeRisk Technologies,Graz,https://at.linkedin.com/jobs/view/support-engi...,demand support engineer qualifications join cl...
9541,It Support Specialist In Italian/English,The Sales Centre,Italy,https://it.linkedin.com/jobs/view/it-support-s...,hire hardworking support specialist passion ...
11416,IT Technician,Blue Arrow,Runcorn,https://uk.linkedin.com/jobs/view/it-technicia...,they recruit technician join expanding team f...
9138,General IT Support Technician,Cognizant,Dublin,https://ie.linkedin.com/jobs/view/general-it-s...,the general support technician required sup...
11122,IT Service Manager,Mission Viejo Auto Collision,Liverpool,https://uk.linkedin.com/jobs/view/it-service-m...,the service manager responsibility service de...


In [28]:
topic_1_resume_2.head()

,Position,Company,City,Link,Clean_description
17178,English Customer Advisor,Cross Border Talents,Lisbon,https://pt.linkedin.com/jobs/view/english-cust...,cross border talents recruiting english custom...
16877,Spanish Customer Service Representatives,Cross Border Talents,Lisbon Metropolitan Area,https://pt.linkedin.com/jobs/view/spanish-cust...,descriptioncross border talents recruiting eng...
16972,English Customer Support Agent,Cross Border Talents,Lisbon,https://pt.linkedin.com/jobs/view/english-cust...,cross border talents recruiting english custom...
16973,Responsável Administrativo e Financeiro,Evollu,Leiria,https://pt.linkedin.com/jobs/view/respons%C3%A...,cross border talents recruiting english custom...
17076,Content development and on,Secretplaces,Cascais,https://pt.linkedin.com/jobs/view/content-deve...,cross border talents recruiting english custom...


Next, we merge the resume with each of the top 3 most relevant topics/fields. This is done for both of our sample resumes.

In [29]:
def merge_job_data_in_topic_and_resume_data(job_data_in_topic, resume_data):
    
    # get job descriptions
    job_data_in_topic.reset_index(inplace=True, drop = True)
    
    # get resume
    row = []
    for i in range(4):
        row.append("Not Applicable")
    row.append(" ".join(resume_data))
    
    # add resume to job descriptions
    job_data_in_topic.loc[job_data_in_topic.shape[0]] = row
    
    #return job_data
    return job_data_in_topic

In [30]:
# merge jobs and resume

# resume 1
merged_jobs_topic_1_resume_1 = merge_job_data_in_topic_and_resume_data(
                                            job_data_in_topic = topic_1_resume_1, 
                                            resume_data = resume_1)

merged_jobs_topic_2_resume_1 = merge_job_data_in_topic_and_resume_data(
                                            job_data_in_topic = topic_2_resume_1, 
                                            resume_data = resume_1) 

merged_jobs_topic_3_resume_1 = merge_job_data_in_topic_and_resume_data(
                                            job_data_in_topic = topic_3_resume_1, 
                                            resume_data = resume_1) 

# resume 2

merged_jobs_topic_1_resume_2 = merge_job_data_in_topic_and_resume_data(
                                            job_data_in_topic = topic_1_resume_2, 
                                            resume_data = resume_2)

merged_jobs_topic_2_resume_2 = merge_job_data_in_topic_and_resume_data(
                                            job_data_in_topic = topic_2_resume_2, 
                                            resume_data = resume_2) 

merged_jobs_topic_3_resume_2 = merge_job_data_in_topic_and_resume_data(
                                            job_data_in_topic = topic_3_resume_2, 
                                            resume_data = resume_2) 

In [31]:
# check that the resume is now in the bottom of the relevant jobs
merged_jobs_topic_1_resume_1.tail()

,Position,Company,City,Link,Clean_description
244,Business Analyst,Smarter Grid Solutions,Glasgow,https://uk.linkedin.com/jobs/view/business-ana...,company overview smarter grid solutions pionee...
245,Enterprise Account Manager,Honeywell,Praha-Dubeč,https://cz.linkedin.com/jobs/view/enterprise-a...,the future what make when join honeywell m...
246,Technical Operations Associate,Expedia Group,Prague,https://cz.linkedin.com/jobs/view/technical-op...,the expedia inc service desk search experie...
247,IT Manager,Michael Page,Glasgow,https://uk.linkedin.com/jobs/view/it-manager-a...,global marine business expanding growing func...
248,Not Applicable,Not Applicable,Not Applicable,Not Applicable,software developer unlimited software develope...


In [32]:
# check that the resume is now in the bottom of the relevant jobs
merged_jobs_topic_1_resume_2.tail()

,Position,Company,City,Link,Clean_description
244,Property Manager,We Build Recruitment,Liverpool,https://uk.linkedin.com/jobs/view/property-man...,liverpool award-winning estate agents based li...
245,English Speaking Help Desk Agent,Cross Border Talents,Lisbon,https://pt.linkedin.com/jobs/view/english-spea...,dynamic motivated person join customer support...
246,Call Center Representative Agent Work From Home,AFG Inc,Glasgow,https://uk.linkedin.com/jobs/view/call-center-...,position call center representative agent wor...
247,Operations Service Delivery Manager,Nets Group,Ballerup,https://dk.linkedin.com/jobs/view/operations-s...,service delivery managers projects automatio...
248,Not Applicable,Not Applicable,Not Applicable,Not Applicable,sidney pereira flight attendant year domestic ...


# Final recommendations

In [33]:
topics.shape

(61, 2)

Everything is now done and we are ready to generate our final recommendations. In the first script we cleaned the resume data and the job data. We then investigated the topic modelling techniques LDA, NMF and Top2Vec to find the best way to group our dataset of jobs into fields. The best performing model appeared to be the Top2Vec with 70 topics. Those topics were then labelled manually. The ones we were not able to label were dropped and topics that received the same name were merged. This reduced our number of topics from 70 to 61. A full visualization of the topics can be observed in the paper or in script 2. We then calculated which top 3 topics/fields a person would be best suited for by the similarity between the resume and each topic. We then found 250 jobs that were the most similar to a particular topic. We did this for each of the top 3 topics.

The final step is now to calculated the similarity between the resume and each of the 250 jobs within each of the three topics. This will be our final output and recommendations for the person with the given resume. We will do this for both of the two resumes in our sample.

In [34]:
def get_recommendations_jobs(topic_data):
    
    cosine_sim = cosine_similarity(topic_data.iloc[: , -1].values.astype(str), 'TFIDF')
    
    # creates a similarity score of every description to the resume
    similarity_scores = sorted(list(enumerate(cosine_sim[-1])), key=lambda x: x[1], reverse=True)
    
    # finds the index of the 20 most similar descriptions
    job_indices = [i[0] for i in similarity_scores[2:20]]
    topic_data[["Position","Company", "City", "Link"]].iloc[job_indices].to_excel(os.getcwd()+'\\recommendations.xlsx')
    
    # output recommendation
    print("Based on your experience we believe you are best suited for the following jobs:")
    return HTML(topic_data[["Position","Company", "City", "Link"]].iloc[job_indices].to_html(render_links=True, escape=False))

### Sample resume 1

In [35]:
get_recommendations_jobs(merged_jobs_topic_1_resume_1)

Based on your experience we believe you are best suited for the following jobs:


,Position,Company,City,Link
60,Graduate Software Developer,My1Login,Glasgow,https://uk.linkedin.com/jobs/view/graduate-software-developer-at-my1login-3027870093?refId=nAh4uo5FyhfhiqKZ95puiQ%3D%3D&trackingId=UTWZAMmhUj9xGneQRstINQ%3D%3D&position=15&pageNum=30&trk=public_jobs_jserp-result_search-card
175,Junior Software Engineer,Deep,Copenhagen,https://dk.linkedin.com/jobs/view/junior-software-engineer-at-deep-3060254882?refId=H0aLOsfhDFpqODpxEPWStA%3D%3D&trackingId=yTODGbXUcHKYd3B4TXGwEA%3D%3D&position=19&pageNum=17&trk=public_jobs_jserp-result_search-card
119,Trainee,Nokia,Espoo,https://fi.linkedin.com/jobs/view/trainee-at-nokia-3028735893?refId=pMjHvG%2BZb9NkWhZgGM0f0g%3D%3D&trackingId=pZxfS1OMfiSbxhpHuR5QGA%3D%3D&position=11&pageNum=33&trk=public_jobs_jserp-result_search-card
203,Quality Engineer,Floww,London,https://uk.linkedin.com/jobs/view/quality-engineer-at-floww-3051499290?refId=7UMA5BImMY8jsMBZKZbHSg%3D%3D&trackingId=lRuzHM1VKNuPFS3SzGeEdg%3D%3D&position=16&pageNum=35&trk=public_jobs_jserp-result_search-card
125,SQL Developer,Barclays,Glasgow,https://uk.linkedin.com/jobs/view/sql-developer-at-barclays-2947336505?refId=7qU93xt%2Ftg12LG0AYlMP5w%3D%3D&trackingId=9ofHd3mBo%2BuCvSx2wSuitQ%3D%3D&position=14&pageNum=1&trk=public_jobs_jserp-result_search-card
195,Junior Web Developer,KPMG Ireland,Dublin,https://ie.linkedin.com/jobs/view/junior-web-developer-at-kpmg-ireland-3040434617?refId=yZUW5z7%2Fp9LZDV5qGhRMKw%3D%3D&trackingId=FvWyR%2FH8TPY3jV%2FnTsmPJw%3D%3D&position=6&pageNum=5&trk=public_jobs_jserp-result_search-card
185,Software Engineer,Verint,Glasgow,https://uk.linkedin.com/jobs/view/software-engineer-at-verint-3047194652?refId=OR5pJ%2BZrsa4mzPSgWTMFlg%3D%3D&trackingId=GcGwMM8arD1DwikGNmM4fQ%3D%3D&position=11&pageNum=37&trk=public_jobs_jserp-result_search-card
187,Robotics Engineer,Seervision AG,Athens,https://gr.linkedin.com/jobs/view/robotics-engineer-at-seervision-ag-2766752170?refId=4vL8KGRgCm0B%2FaF7rGbMJQ%3D%3D&trackingId=Hn5AUHzcXwZMDftnpk5KHA%3D%3D&position=17&pageNum=39&trk=public_jobs_jserp-result_search-card
163,Customer Support Specialist,UserTesting,Edinburgh,https://uk.linkedin.com/jobs/view/customer-support-specialist-at-usertesting-3039259438?refId=cdoZzIZKowtM%2F6TO6qbeNA%3D%3D&trackingId=O9LkQMsye7mxDm24HD13cQ%3D%3D&position=21&pageNum=5&trk=public_jobs_jserp-result_search-card
169,Front,Google,Munich,https://de.linkedin.com/jobs/view/front-end-software-engineer-google-account-at-google-3045248473?refId=iCCPUFnqr6LOiwN4fgZAug%3D%3D&trackingId=0lY4goAgt%2B7K1TMfI46Ukg%3D%3D&position=19&pageNum=8&trk=public_jobs_jserp-result_search-card


In [36]:
get_recommendations_jobs(merged_jobs_topic_2_resume_1)

Based on your experience we believe you are best suited for the following jobs:


,Position,Company,City,Link
180,Java Developer,Barclays,Glasgow,https://uk.linkedin.com/jobs/view/java-developer-at-barclays-2927350844?refId=Wwu1eSkGeekk5BRG5Sy0cw%3D%3D&trackingId=Ss11pINHT1JQUv%2FuMepshg%3D%3D&position=21&pageNum=8&trk=public_jobs_jserp-result_search-card
167,Software Engineer,Honeywell,Dublin 1,https://ie.linkedin.com/jobs/view/software-engineer-at-honeywell-3069129084?refId=H74XrUZuHFKDe6mBt%2FFehg%3D%3D&trackingId=6M%2B4NJqzhJn1JaCeQXR57w%3D%3D&position=25&pageNum=17&trk=public_jobs_jserp-result_search-card
102,Junior Software Engineer,Verimatrix,Glasgow,https://uk.linkedin.com/jobs/view/junior-software-engineer-at-verimatrix-3065057858?refId=4HaASpTOkQllpuJTF8p7sg%3D%3D&trackingId=Oe18HDQFKwxZWumGrB07kw%3D%3D&position=25&pageNum=13&trk=public_jobs_jserp-result_search-card
63,Trainee,Nokia,Espoo,https://fi.linkedin.com/jobs/view/trainee-at-nokia-3028735893?refId=pMjHvG%2BZb9NkWhZgGM0f0g%3D%3D&trackingId=pZxfS1OMfiSbxhpHuR5QGA%3D%3D&position=11&pageNum=33&trk=public_jobs_jserp-result_search-card
114,Java Developer,Morgan Stanley,Glasgow,https://uk.linkedin.com/jobs/view/java-developer-at-morgan-stanley-2856890309?refId=IDf0%2BOn1TAqOkbNnla6C9w%3D%3D&trackingId=2tj62roNuHWKjpAla6GBfA%3D%3D&position=22&pageNum=39&trk=public_jobs_jserp-result_search-card
104,Software Engineer,BBC,Glasgow,https://uk.linkedin.com/jobs/view/software-engineer-at-bbc-3047182788?refId=r0wGCAGJaob%2Faz20Sn%2FMKQ%3D%3D&trackingId=8Z2YZyjsHLplm0x1hClMUA%3D%3D&position=11&pageNum=4&trk=public_jobs_jserp-result_search-card
197,SQL Developer,Barclays,Glasgow,https://uk.linkedin.com/jobs/view/sql-developer-at-barclays-2947336505?refId=7qU93xt%2Ftg12LG0AYlMP5w%3D%3D&trackingId=9ofHd3mBo%2BuCvSx2wSuitQ%3D%3D&position=14&pageNum=1&trk=public_jobs_jserp-result_search-card
103,Java Developer,Morgan Stanley,Glasgow,https://uk.linkedin.com/jobs/view/java-developer-at-morgan-stanley-3050768353?refId=My6sNR8pbVAaOvw4XavZtg%3D%3D&trackingId=9RmCOxGVMvUAE2AC5l%2FslA%3D%3D&position=8&pageNum=20&trk=public_jobs_jserp-result_search-card
158,Entry Level Front,Keyvoto,Athens,https://gr.linkedin.com/jobs/view/entry-level-front-end-developer-at-keyvoto-3003688970?refId=4cPCDU3rWBLAdSMSjYrWfg%3D%3D&trackingId=4MOFvLgJ289ir0kM7%2FBamg%3D%3D&position=14&pageNum=23&trk=public_jobs_jserp-result_search-card
218,Software Engineer,Verint,Glasgow,https://uk.linkedin.com/jobs/view/software-engineer-at-verint-3047194652?refId=OR5pJ%2BZrsa4mzPSgWTMFlg%3D%3D&trackingId=GcGwMM8arD1DwikGNmM4fQ%3D%3D&position=11&pageNum=37&trk=public_jobs_jserp-result_search-card


In [37]:
get_recommendations_jobs(merged_jobs_topic_3_resume_1)

Based on your experience we believe you are best suited for the following jobs:


,Position,Company,City,Link
197,Junior QA Technician,Zeigo,London,https://uk.linkedin.com/jobs/view/junior-qa-technician-at-zeigo-3053611171?refId=i18SxyTmiFUClF6BvCSfzw%3D%3D&trackingId=xmSoOhpCz2K%2FqIVHJNiNMg%3D%3D&position=3&pageNum=19&trk=public_jobs_jserp-result_search-card
136,Mechanical Engineer,Welltec,Allerød,https://dk.linkedin.com/jobs/view/mechanical-engineer-at-welltec-3047760767?refId=5gDwSK8m4%2B4BtcTzLgEx6w%3D%3D&trackingId=6qGUiG%2BaNnR3tgugN8GbbQ%3D%3D&position=3&pageNum=21&trk=public_jobs_jserp-result_search-card
121,Senior Mechanical Design Engineer – Robotic,Medtronic,Wessling,https://de.linkedin.com/jobs/view/senior-mechanical-design-engineer-m-f-d-%E2%80%93-robotic-assisted-surgery-research-and-development-wessling-germany-at-medtronic-3049961819?refId=PLapX3ZxenUeqlOSqVXfRA%3D%3D&trackingId=v4uhhTCxw7vMhpHXO7V67w%3D%3D&position=6&pageNum=26&trk=public_jobs_jserp-result_search-card
33,Quality Control Analyst,Johnson & Johnson,Cork,https://ie.linkedin.com/jobs/view/quality-control-analyst-at-johnson-johnson-3028744611?refId=vUudO8KqOnKDqGgprO2L4g%3D%3D&trackingId=q%2BG%2Fpz02pKQVwIFiW6TMyA%3D%3D&position=10&pageNum=22&trk=public_jobs_jserp-result_search-card
211,Associate Software Engineer,CliniSys,Glasgow,https://uk.linkedin.com/jobs/view/associate-software-engineer-at-clinisys-2941506253?refId=ltcx2cdthT0ptEtOm7hofw%3D%3D&trackingId=7VN1sC5aSpglfp3Gpv1%2FBg%3D%3D&position=12&pageNum=36&trk=public_jobs_jserp-result_search-card
81,Software Developer,BioClavis Ltd,Glasgow,https://uk.linkedin.com/jobs/view/software-developer-at-bioclavis-ltd-3023506669?refId=pzgEi5xo3muJLATXJFwmwQ%3D%3D&trackingId=qLX24pZm7kUr7hcVr8Y0OQ%3D%3D&position=10&pageNum=35&trk=public_jobs_jserp-result_search-card
84,Medical Device Developer,Novo Nordisk,Hillerød,https://dk.linkedin.com/jobs/view/medical-device-developer-systems-engineer-at-novo-nordisk-3051240362?refId=2sIIMgPBP6wfJAY2TvSIiA%3D%3D&trackingId=%2Fv9ZS3X6EYtf8Fvfdtjm%2Fg%3D%3D&position=16&pageNum=14&trk=public_jobs_jserp-result_search-card
65,Assistant Country Manager,Cureety,Milan,https://it.linkedin.com/jobs/view/assistant-country-manager-at-cureety-3056734010?refId=lEGtji8tJSNJZMWYB3sByg%3D%3D&trackingId=PLdONJau%2FHYekDRtvTYMPA%3D%3D&position=23&pageNum=12&trk=public_jobs_jserp-result_search-card
29,R&D Test Engineers across seniority levels,Novo Nordisk,Hillerød,https://dk.linkedin.com/jobs/view/r-d-test-engineers-across-seniority-levels-at-novo-nordisk-3051241275?refId=kpS5rlYwks3JHlSKX3bxtg%3D%3D&trackingId=0QubHPK1wIJypIokEhhKeA%3D%3D&position=6&pageNum=2&trk=public_jobs_jserp-result_search-card
24,Chemistry Intern,Novo Nordisk,Måløv,https://dk.linkedin.com/jobs/view/chemistry-intern-at-novo-nordisk-3049288249?refId=0uWgexKmELgZ0Sciwk7tPw%3D%3D&trackingId=fRpu06%2B3gw8JKpRVyrSc1Q%3D%3D&position=4&pageNum=0&trk=public_jobs_jserp-result_search-card


### Sample resume 2

In [38]:
get_recommendations_jobs(merged_jobs_topic_1_resume_2)

Based on your experience we believe you are best suited for the following jobs:


,Position,Company,City,Link
72,Flight Operations Administrator,TUI,Zaventem,https://be.linkedin.com/jobs/view/flight-operations-administrator-at-tui-3043483153?refId=Hit58RpgQ9bzUjzb9UdHDA%3D%3D&trackingId=PqBYjxyMZSB8k2o8ki39Mw%3D%3D&position=3&pageNum=12&trk=public_jobs_jserp-result_search-card
159,Customer Services Agent,Aer Lingus,Dublin,https://ie.linkedin.com/jobs/view/customer-services-agent-fixed-term-contract-at-aer-lingus-3045221097?refId=7JU4EWkv%2BkrUveWzA3zMqQ%3D%3D&trackingId=uZ8tVXUkZisYqPHXv3DbXQ%3D%3D&position=2&pageNum=6&trk=public_jobs_jserp-result_search-card
27,Italian Customer Service Agent In Lisbon,Exodius,Lisbon,https://pt.linkedin.com/jobs/view/italian-customer-service-agent-in-lisbon-at-exodius-3053716721?refId=f5uNCwMo3uB6pBpkDp2cZQ%3D%3D&trackingId=%2FazMHpk%2FRBoFa1g3xNzbkw%3D%3D&position=4&pageNum=32&trk=public_jobs_jserp-result_search-card
1,Spanish Customer Service Representatives,Cross Border Talents,Lisbon Metropolitan Area,https://pt.linkedin.com/jobs/view/spanish-customer-service-representatives-at-cross-border-talents-3083629131?refId=V67Ep2vuKQ7Xp9tI03i%2Frw%3D%3D&trackingId=sLYRt2WufjOjk7xdoyDNZQ%3D%3D&position=7&pageNum=0&trk=public_jobs_jserp-result_search-card
8,Spanish Customer Service Representatives,Cross Border Talents,Lisbon Metropolitan Area,https://pt.linkedin.com/jobs/view/spanish-customer-service-representatives-at-cross-border-talents-3083629131?refId=V67Ep2vuKQ7Xp9tI03i%2Frw%3D%3D&trackingId=sLYRt2WufjOjk7xdoyDNZQ%3D%3D&position=7&pageNum=0&trk=public_jobs_jserp-result_search-card
9,Customer Service Representative,Cross Border Talents,Lisbon,https://pt.linkedin.com/jobs/view/customer-service-representative-at-cross-border-talents-3087178640?refId=V67Ep2vuKQ7Xp9tI03i%2Frw%3D%3D&trackingId=%2FzMeVr2cR5FFeUrW5w0jnw%3D%3D&position=8&pageNum=0&trk=public_jobs_jserp-result_search-card
178,Pilot Rostering Analyst,Ryanair - Europe's Favourite Airline,Dublin,https://ie.linkedin.com/jobs/view/pilot-rostering-analyst-at-ryanair-europe-s-favourite-airline-3049408808?refId=nDzLU%2FifwGNRIy%2F0OrAVsw%3D%3D&trackingId=3jhB4en8lFBqkw3kZGRBmA%3D%3D&position=24&pageNum=16&trk=public_jobs_jserp-result_search-card
133,Sales and Station Agent,Turkish Airlines,Munich,https://de.linkedin.com/jobs/view/sales-and-station-agent-at-turkish-airlines-3040859788?refId=THKV3WglWQ84u2Y9OrKg6g%3D%3D&trackingId=1VzM5UU3qtMCzpMk4w71tg%3D%3D&position=18&pageNum=6&trk=public_jobs_jserp-result_search-card
186,Insurance Customer Service/Sales Advisor,Office Angels,Edinburgh,https://uk.linkedin.com/jobs/view/insurance-customer-service-sales-advisor-at-office-angels-3067506978?refId=pvTBOqRBlSiH53%2FTQc9D4Q%3D%3D&trackingId=WtXTeptYtryRBkqPdgWpwg%3D%3D&position=13&pageNum=14&trk=public_jobs_jserp-result_search-card
137,Sales and Station Agent,Turkish Airlines,Berlin,https://de.linkedin.com/jobs/view/sales-and-station-agent-at-turkish-airlines-3052567215?refId=u0vzQ%2Fbt6%2BbB%2FbQ3YhsFjA%3D%3D&trackingId=ntg0SkC02gbVi9Oe5k4Pkw%3D%3D&position=14&pageNum=25&trk=public_jobs_jserp-result_search-card


In [39]:
get_recommendations_jobs(merged_jobs_topic_2_resume_2)

Based on your experience we believe you are best suited for the following jobs:


,Position,Company,City,Link
124,Magazziniere,Home Fitness Center,Sona,https://it.linkedin.com/jobs/view/magazziniere-at-home-fitness-center-3060995971?refId=fOSviZjtF3tXQuEWX1nNoA%3D%3D&trackingId=zztRL792rzGbI%2Fn9azPW%2Bg%3D%3D&position=19&pageNum=1&trk=public_jobs_jserp-result_search-card
160,Assistant Hotel Operations Manager,Bahar Boutique Hotel,Thessaloniki Metropolitan Area,https://gr.linkedin.com/jobs/view/assistant-hotel-operations-manager-at-bahar-boutique-hotel-2992704443?refId=JXdrPk6Fsho7arqTpXXNFQ%3D%3D&trackingId=tdDY30vlcGDHdRQiaGTarg%3D%3D&position=14&pageNum=37&trk=public_jobs_jserp-result_search-card
184,RIVER,Viking,Basel,https://ch.linkedin.com/jobs/view/river-guest-services-agent-at-viking-3057373221?refId=JazcFWyiuh%2BACDOB5p28yw%3D%3D&trackingId=HDtahf0gDZ5Gw5BeB%2FFcaA%3D%3D&position=9&pageNum=9&trk=public_jobs_jserp-result_search-card
1,Night Manager,Hilton,Antwerpområdet,https://be.linkedin.com/jobs/view/night-manager-emea-at-hilton-3047640455?refId=YOHCJ9WNwJpgJ70I7CeRng%3D%3D&trackingId=Hf7c%2FlMKLtp0uZFXjbNSUQ%3D%3D&position=10&pageNum=36&trk=public_jobs_jserp-result_search-card
187,Sales & Reservations Assistant,The Travel Corporation,Edinburgh,https://uk.linkedin.com/jobs/view/sales-reservations-assistant-at-the-travel-corporation-3069540110?refId=vz5TxbP0ApshuA%2FGVTkhAA%3D%3D&trackingId=LzCqqf8%2FPeL%2Fp%2FHP1j8H7w%3D%3D&position=16&pageNum=10&trk=public_jobs_jserp-result_search-card
135,Corporate & M.I.C.E. Consultant,Mideast Travel Worldwide,Athens,https://gr.linkedin.com/jobs/view/corporate-m-i-c-e-consultant-at-mideast-travel-worldwide-3046853762?refId=dZm94Q2tCEL51xbHJ%2FupAw%3D%3D&trackingId=rprL%2FyT4L7GpcnfI1ASkvg%3D%3D&position=14&pageNum=6&trk=public_jobs_jserp-result_search-card
213,Hotel Night Receptionist,The University of Edinburgh,Edinburgh,https://uk.linkedin.com/jobs/view/hotel-night-receptionist-at-the-university-of-edinburgh-3071003220?refId=Akbcfj1CDOWXYGqwJV0qwg%3D%3D&trackingId=uhdVQHfPxr32LTkz8gnEUA%3D%3D&position=5&pageNum=18&trk=public_jobs_jserp-result_search-card
195,Client Experience Host,Goldsmiths,Liverpool,https://uk.linkedin.com/jobs/view/client-experience-host-30-hours-at-goldsmiths-3039230488?refId=v5E6VCghrlvFkpJzr0kARQ%3D%3D&trackingId=eBqV1yGT9afuONhcD6mnZw%3D%3D&position=25&pageNum=16&trk=public_jobs_jserp-result_search-card
240,MISSISSIPPI,Viking,Basel,https://ch.linkedin.com/jobs/view/mississippi-guest-services-staff-at-viking-3055967944?refId=B0sKJHOXLdkElugJX1lmig%3D%3D&trackingId=kEskk4M%2F5SmP%2FgHWvr%2B2gw%3D%3D&position=21&pageNum=18&trk=public_jobs_jserp-result_search-card
50,Lodging Partner Associate I,Expedia Group,Prague,https://cz.linkedin.com/jobs/view/lodging-partner-associate-i-spanish-and-english-speaking-at-expedia-group-2955176640?refId=nQRwnc1Tnx%2BMxuAZ%2B6wRQA%3D%3D&trackingId=6ne4BUmf8rTB7fiHRP%2FSqw%3D%3D&position=6&pageNum=3&trk=public_jobs_jserp-result_search-card


In [40]:
get_recommendations_jobs(merged_jobs_topic_3_resume_2)

Based on your experience we believe you are best suited for the following jobs:


,Position,Company,City,Link
152,Feedback and Insight Officer,MCCY ltd,Edinburgh,https://uk.linkedin.com/jobs/view/feedback-and-insight-officer-at-mccy-ltd-3062224119?refId=OmrGqpZrPVRk%2FHtlsQW3Jw%3D%3D&trackingId=C7SO0uxGdpAkxBoa%2FQxttQ%3D%3D&position=22&pageNum=17&trk=public_jobs_jserp-result_search-card
179,HR Specialist,"BPO Europe, s.r.o.",Prague,https://cz.linkedin.com/jobs/view/hr-specialist-microsoft-at-bpo-europe-s-r-o-2928395544?refId=kYFKizHXeZ123WzAc1ulzw%3D%3D&trackingId=Ya20wMKJKuDmhRKXsZCscA%3D%3D&position=19&pageNum=21&trk=public_jobs_jserp-result_search-card
122,Country Manager,HR Talent House,Norway,https://no.linkedin.com/jobs/view/country-manager-at-hr-talent-house-3030483419?refId=QxEW%2BLnY2A1EqA%2BQbQxbuQ%3D%3D&trackingId=hW%2BBnzRmWTm2%2B8kDV94ekQ%3D%3D&position=3&pageNum=13&trk=public_jobs_jserp-result_search-card
110,Teaching Operations Assistant CZ,British Council,Prague,https://cz.linkedin.com/jobs/view/teaching-operations-assistant-cz-at-british-council-3058317828?refId=nQRwnc1Tnx%2BMxuAZ%2B6wRQA%3D%3D&trackingId=mo9XYzqiDSj%2FSNggQRLs%2BA%3D%3D&position=4&pageNum=3&trk=public_jobs_jserp-result_search-card
197,Intern: Concept Creative,Cosmonauts & Kings,Berlin,https://de.linkedin.com/jobs/view/intern-concept-creative-m-f-d-at-cosmonauts-kings-3047661906?refId=0%2BxUC51gmiSuuKjueb%2FN0Q%3D%3D&trackingId=xocOnUmTMR8lRTtoFzOQUg%3D%3D&position=4&pageNum=37&trk=public_jobs_jserp-result_search-card
191,Retail Manager,F10 Human Resource,Helsinki Metropolitan Area,https://fi.linkedin.com/jobs/view/retail-manager-at-f10-human-resource-3068290669?refId=cWJhboMFy5lLMofNoGyKyA%3D%3D&trackingId=92F0ojxU%2BgHqw4SXkMR1TA%3D%3D&position=17&pageNum=37&trk=public_jobs_jserp-result_search-card
140,REGIONAL SERVICE SALES LEADER,INNIO Group,Jenbach,https://at.linkedin.com/jobs/view/regional-service-sales-leader-middle-east-africa-turkey-south-asia-at-innio-group-3050794817?refId=IiQv0crwZ7JkNiKLzOjsKA%3D%3D&trackingId=KowXMjEP29MYAQEzqImhIg%3D%3D&position=5&pageNum=22&trk=public_jobs_jserp-result_search-card
190,Administrator,Adecco,Liverpool,https://uk.linkedin.com/jobs/view/administrator-european-case-worker-liverpool-eo-grade-at-adecco-3044947903?refId=I91PI7Z6jo7MX3q7BbHELw%3D%3D&trackingId=lg6ZFUC4ZLLvC%2FNPgcWXmA%3D%3D&position=11&pageNum=3&trk=public_jobs_jserp-result_search-card
91,Office Manager,AgroBiogel GmbH,Tulln An Der Donau,https://at.linkedin.com/jobs/view/office-manager-at-agrobiogel-gmbh-3064748495?refId=IZVWxoPcH4ZW09ywwjLvhg%3D%3D&trackingId=gL9EidHTC6NYOMgdsUXIpw%3D%3D&position=10&pageNum=0&trk=public_jobs_jserp-result_search-card
186,Regional Service Sales Leader,INNIO Group,Jenbach,https://at.linkedin.com/jobs/view/regional-service-sales-leader-middle-east-africa-turkey-south-asia-m-f-d-at-innio-group-3049441058?refId=q5cosjGsIxzEYt3c55eAbA%3D%3D&trackingId=5AETHwKRory609kZeuIYTQ%3D%3D&position=9&pageNum=36&trk=public_jobs_jserp-result_search-card


# Compare recommendations to baseline model

### Generating recommendations using TFIDF and cosine similarity between resume and jobs (without using topic modelling)

In [41]:
# add resume data to the bottom of the job data df

def merge_job_data_resume_data(job_data, resume_data):
    
    # get job descriptions
    job_data.reset_index(inplace=True, drop = True)
 
    # get resume
    row = []
    for i in range(12):
        row.append("Not Applicable")
    row.append(" ".join(resume_data))
    
    # add resume to job descriptions
    job_data.loc[job_data.shape[0]] = row
    
    #return job_data
    return job_data

In [42]:
# generate recommendations

def get_recommendations_jobs(resume):
    
    merged = merge_job_data_resume_data(job_data = job_data,resume_data = resume)
    
    cosine_sim = cosine_similarity(merged.iloc[: , -1].values.astype(str), 'TFIDF')
    
    # creates a similarity score of every description to the resume
    similarity_scores = sorted(list(enumerate(cosine_sim[-1])), key=lambda x: x[1], reverse=True)
    
    # finds the index of the 20 most similar descriptions
    job_indices = [i[0] for i in similarity_scores[2:20]]
    
    # save data (optional)
    job_data[["Position","Company", "City", "Link"]].iloc[job_indices].to_excel(os.getcwd()+'\\recommendations.xlsx')
    
    # output recommendation
    print("Based on your experience we believe you are best suited for the following jobs:")
    return HTML(job_data[["Position","Company", "City", "Link"]].iloc[job_indices].to_html(render_links=True, escape=False))

In [43]:
get_recommendations_jobs(resume_1)

Based on your experience we believe you are best suited for the following jobs:


,Position,Company,City,Link
2524,Finance Analyst Legal Entity Accounting.,Johnson & Johnson,Prague,https://cz.linkedin.com/jobs/view/finance-analyst-legal-entity-accounting-at-johnson-johnson-3028627461?refId=hbSi1ET5ZvqTE2%2B2I0WQoQ%3D%3D&trackingId=FDrlXSUAjfydZyMwmS0IFg%3D%3D&position=24&pageNum=34&trk=public_jobs_jserp-result_search-card
2517,Global Learning Admin Team Lead EMEA.,Johnson & Johnson,Prague,https://cz.linkedin.com/jobs/view/global-learning-admin-team-lead-emea-at-johnson-johnson-3023980578?refId=hbSi1ET5ZvqTE2%2B2I0WQoQ%3D%3D&trackingId=k9I5pHUnUypmP8aFn90ALQ%3D%3D&position=15&pageNum=34&trk=public_jobs_jserp-result_search-card
2220,Inventory Analyst.,Johnson & Johnson,Prague,https://cz.linkedin.com/jobs/view/inventory-analyst-at-johnson-johnson-2931755699?refId=FtGLFT57rrl%2FnhWL8l7lSw%3D%3D&trackingId=qGB07EzyLKUiaAEzGJCdAQ%3D%3D&position=3&pageNum=19&trk=public_jobs_jserp-result_search-card
3544,Pricing Analyst.,Johnson & Johnson,Helsinki,https://fi.linkedin.com/jobs/view/pricing-analyst-at-johnson-johnson-2974740880?refId=47iLy8Lro1WFvx%2F8u1BHaQ%3D%3D&trackingId=CFOqO37JgDQCXaflVbSVBg%3D%3D&position=6&pageNum=14&trk=public_jobs_jserp-result_search-card
2525,Cash Collection & Cash Application Representative.,Johnson & Johnson,Prague,https://cz.linkedin.com/jobs/view/cash-collection-cash-application-representative-at-johnson-johnson-3060567191?refId=hbSi1ET5ZvqTE2%2B2I0WQoQ%3D%3D&trackingId=6rrQHHAx3jfETT%2BHoyLw1A%3D%3D&position=25&pageNum=34&trk=public_jobs_jserp-result_search-card
2221,Customer Service Representative Nordics,Ortho Clinical Diagnostics,Prague,https://cz.linkedin.com/jobs/view/customer-service-representative-nordics-at-ortho-clinical-diagnostics-3028081668?refId=FtGLFT57rrl%2FnhWL8l7lSw%3D%3D&trackingId=DQOiwdi9s4f8z5xWGOJVfA%3D%3D&position=5&pageNum=19&trk=public_jobs_jserp-result_search-card
2380,HR Contact Centre Specialist,Johnson & Johnson,Prague,https://cz.linkedin.com/jobs/view/hr-contact-centre-specialist-with-french-at-johnson-johnson-3048353266?refId=sSnB50bdUqjtoHQRW7AIag%3D%3D&trackingId=yKDfbYrvaVhy5Ps6qt%2BYlA%3D%3D&position=15&pageNum=27&trk=public_jobs_jserp-result_search-card
2187,Recruitment & Onboarding Specialist,Johnson & Johnson,Prague,https://cz.linkedin.com/jobs/view/recruitment-onboarding-specialist-with-italian-at-johnson-johnson-3063571930?refId=vf738Da1Vf8vb2q3AMIONA%3D%3D&trackingId=XUcP3lRcXIikfBj2e%2BEnZA%3D%3D&position=9&pageNum=17&trk=public_jobs_jserp-result_search-card
13005,Apprenticeship Program Specialist.,Johnson & Johnson,Zug,https://ch.linkedin.com/jobs/view/apprenticeship-program-specialist-at-johnson-johnson-3059201373?refId=VYSWtNwtjikum8LVFkb6MA%3D%3D&trackingId=4p%2FVwyUxfkiifNorGpYmfg%3D%3D&position=5&pageNum=14&trk=public_jobs_jserp-result_search-card
12107,Nordic Sales Analyst Graduate.,Johnson & Johnson,Solna,https://se.linkedin.com/jobs/view/nordic-sales-analyst-graduate-at-johnson-johnson-2902803979?refId=YBiMr%2Bwd123Do5AgrQ3zBA%3D%3D&trackingId=ePEI8PPN29pYyVBasAdKFw%3D%3D&position=15&pageNum=5&trk=public_jobs_jserp-result_search-card


In [44]:
get_recommendations_jobs(resume_2)

Based on your experience we believe you are best suited for the following jobs:


,Position,Company,City,Link
6089,Operations Assistant,Gainjet Ireland (Official),Shannon,https://ie.linkedin.com/jobs/view/operations-assistant-at-gainjet-ireland-official-3035916523?refId=jqLCCDI6dExZVoQgxOsLBg%3D%3D&trackingId=CG5NCKbt2uemRBDwHatxMg%3D%3D&position=2&pageNum=18&trk=public_jobs_jserp-result_search-card
132,Flight Dispatcher,GlobeAir,Hörsching,https://at.linkedin.com/jobs/view/flight-dispatcher-at-globeair-3045354577?refId=EyP8YhnmXO6xfXFUQHU69Q%3D%3D&trackingId=eoHoBI%2BYkPGDtlIKAxCzaQ%3D%3D&position=20&pageNum=7&trk=public_jobs_jserp-result_search-card
9252,Content Editor,Tourism Group International,Amsterdam,https://nl.linkedin.com/jobs/view/content-editor-at-tourism-group-international-3092667409?refId=L2drXHznmcI2VgQ9AOmoJQ%3D%3D&trackingId=w3GXI0kScVf37GJ07NwtfA%3D%3D&position=5&pageNum=0&trk=public_jobs_jserp-result_search-card
9733,Passenger Traffic Operations Employee,Korean Air,Schiphol,https://nl.linkedin.com/jobs/view/passenger-traffic-operations-employee-at-korean-air-3092312462?refId=L2drXHznmcI2VgQ9AOmoJQ%3D%3D&trackingId=GNSY1xLHu69H%2FEox61RwWw%3D%3D&position=4&pageNum=0&trk=public_jobs_jserp-result_search-card
6063,Pilot Rostering Analyst,Ryanair - Europe's Favourite Airline,Dublin,https://ie.linkedin.com/jobs/view/pilot-rostering-analyst-at-ryanair-europe-s-favourite-airline-3049408808?refId=nDzLU%2FifwGNRIy%2F0OrAVsw%3D%3D&trackingId=3jhB4en8lFBqkw3kZGRBmA%3D%3D&position=24&pageNum=16&trk=public_jobs_jserp-result_search-card
11008,Italian Customer Service Agent In Lisbon,Exodius,Lisbon,https://pt.linkedin.com/jobs/view/italian-customer-service-agent-in-lisbon-at-exodius-3053716721?refId=f5uNCwMo3uB6pBpkDp2cZQ%3D%3D&trackingId=%2FazMHpk%2FRBoFa1g3xNzbkw%3D%3D&position=4&pageNum=32&trk=public_jobs_jserp-result_search-card
67,Management trainee: Flight Operations Training and Safety,Ryanair - Europe's Favourite Airline,Schwechat,https://at.linkedin.com/jobs/view/management-trainee-flight-operations-training-and-safety-at-ryanair-europe-s-favourite-airline-3058132076?refId=cGH5uAThzl55KhCv3A7EnA%3D%3D&trackingId=iMpTfEFPeLHqbsb59%2F6L2Q%3D%3D&position=19&pageNum=3&trk=public_jobs_jserp-result_search-card
5828,Customer Services Agent,Aer Lingus,Dublin,https://ie.linkedin.com/jobs/view/customer-services-agent-fixed-term-contract-at-aer-lingus-3045221097?refId=7JU4EWkv%2BkrUveWzA3zMqQ%3D%3D&trackingId=uZ8tVXUkZisYqPHXv3DbXQ%3D%3D&position=2&pageNum=6&trk=public_jobs_jserp-result_search-card
62,Customer Experience Specialist,GlobeAir,Hörsching,https://at.linkedin.com/jobs/view/customer-experience-specialist-at-globeair-3066692371?refId=cGH5uAThzl55KhCv3A7EnA%3D%3D&trackingId=rctOvQr4TEPQigHiIyQBlA%3D%3D&position=11&pageNum=3&trk=public_jobs_jserp-result_search-card
5762,Operations Graduate Programmes,Ryanair - Europe's Favourite Airline,Dublin,https://ie.linkedin.com/jobs/view/operations-graduate-programmes-2022-at-ryanair-europe-s-favourite-airline-3036816842?refId=6QL2Vc0QLOs409YA2qjaYw%3D%3D&trackingId=2eQ5aLfN6u6QL0VFv%2BzpbQ%3D%3D&position=5&pageNum=3&trk=public_jobs_jserp-result_search-card
